## Charts Exploratory

In [ ]:
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS
from shared_utils import portfolio_utils, rt_dates, publish_utils
readable_dict = OmegaConf.load("readable2.yml")
credentials, project = google.auth.default()

import _report_operator_visuals
import _report_route_dir_visuals
import _scheduled_service_hour_visuals

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)


def formatted(number):
    return "{:,}".format(number)

In [ ]:
# Comment out and leave this cell right below pandas
# portfolio_name = "Curry Public Transit"
portfolio_name =  "Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"

In [ ]:
ROUTE_DIR_MONTH_FILE = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp_report

In [ ]:
route_dir_month_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_MONTH_FILE}.parquet",
    filters=[[("Portfolio Organization Name", "==", portfolio_name)]],
)

In [ ]:
OG_ROUTE_DIR_MONTH_FILE = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp
og_month_route_dir_df = pd.read_parquet(
        f"{RT_SCHED_GCS}{OG_ROUTE_DIR_MONTH_FILE}.parquet",
    filters=[[("portfolio_organization_name", "==", portfolio_name)]]
    )

In [ ]:
ROUTE_DIR_QTR_FILE = 'digest/quarterly_schedule_vp_metrics_report'
route_dir_qtr_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_QTR_FILE}.parquet",
    filters=[[("Portfolio Organization Name", "==", portfolio_name)]],
)


### Some of the names are Route 1 Route 1, Route 16 Route 16...See what's up.
* When we combine `route_short_name` and `route_long_name` sometimes the values are the same for both columns so we get repeated values.

In [ ]:
import yaml
with open("../_shared_utils/shared_utils/portfolio_organization_name.yml", "r") as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [ ]:
CLEANED_ROUTE_NAMING = GTFS_DATA_DICT.schedule_tables.route_identification

clean_route_names = pd.read_parquet(f"{SCHED_GCS}{CLEANED_ROUTE_NAMING}.parquet").pipe(
    portfolio_utils.standardize_portfolio_organization_names,
    PORTFOLIO_ORGANIZATIONS_DICT,
)

In [ ]:
clean_route_names.columns

In [ ]:
clean_route_names= clean_route_names.loc[clean_route_names.portfolio_organization_name == portfolio_name]

In [ ]:
clean_route_names[["recent_combined_name", "route_long_name","route_short_name"]].drop_duplicates()

### Lots of duplicate rows are popping up...Fix that

In [ ]:
one_test_route = route_dir_month_df.loc[(route_dir_month_df.Date == '2025-05-14T00:00:00.000000000') & (route_dir_month_df.Route =='89 80-89 Coastal Express')]

In [ ]:
len(one_test_route)

In [ ]:
len(one_test_route.drop_duplicates())

In [ ]:
one_test_route.drop_duplicates()

## 

In [ ]:
display(_report_route_dir_visuals.route_filter(route_dir_qtr_df,route_dir_month_df))